In [36]:
import time
import numpy as np
from numba import njit
from typing import Callable, List, Any, Optional
import sigkernel

import numpy as np
import pandas as pd
import sklearn.metrics
from typing import List, Optional, Dict, Set, Callable, Any
from joblib import Memory, Parallel, delayed
import tslearn
import tslearn.metrics
from tslearn.datasets import UCR_UEA_datasets
from scipy.interpolate import interp1d
from numba import njit
import numba as nb
import pickle

from experiments.experiment_code import print_dataset_stats, run_all_kernels

from models.signature import streams_to_sigs
from models.kernels import linear_kernel_gram, pairwise_kernel_gram
from models.conformance import stream_to_torch

def calc_iisig_kernel(X, Y, order):
    sig_X, sig_Y = streams_to_sigs([X,Y], order, disable_tqdm=True)
    dot = 1 + np.dot(sig_X, sig_Y)
    return dot


def case_sig_pde(train:List[np.ndarray], 
                 test:List[np.ndarray], 
                 dyadic_order:int = 5,
                 static_kernel = sigkernel.LinearKernel(),
                 n_jobs:int = 1,
                 verbose:bool = False,
                ):
    """Calculates the signature kernel gram matrices of the train and test.
    Train and test are lists of possibly variable length multidimension 
    time series of shape (T_i, d)"""
    sig_kernel = sigkernel.SigKernel(static_kernel, dyadic_order)
    kernel = lambda s1, s2 : sig_kernel.compute_kernel(
                                stream_to_torch(s1), 
                                stream_to_torch(s2)).numpy()[0]
    vv_gram = pairwise_kernel_gram(train, train, kernel, sym=True, n_jobs=n_jobs, verbose=verbose)
    uv_gram = pairwise_kernel_gram(test, train, kernel, sym=False, n_jobs=n_jobs, verbose=verbose)
    return vv_gram, uv_gram


def calc_sigpde_kernel(X,Y):
    dyadic_order = 3
    T, d = X.shape
    static_kernel = sigkernel.LinearKernel(scale=1) #TODO change here
    vv, uv = case_sig_pde([X], [Y], dyadic_order, static_kernel)
    return uv[0,0]


# def calc_ksig_kernel(X,Y, order):
#     import ksig
#     static_kernel = ksig.static.kernels.LinearKernel() 
#     sig_kernel = ksig.kernels.SignatureKernel(n_levels=order, order=1, static_kernel=static_kernel, normalize=False)
#     dot = sig_kernel(np.array([X,X]), np.array([Y,Y]))[0,0]
#     return dot


def trunc_sig_kernel(s1:np.ndarray, 
                    s2:np.ndarray, 
                    order:int, #order is truncation level of the signature
                    static_kernel_gram:Callable,
                    only_last:bool = True,

                    ):
    """s1 and s2 are time series of shape (T_i, d)"""
    T,d = s1.shape
    K = static_kernel_gram(s1, s2)
    nabla = K[1:, 1:] + K[:-1, :-1] - K[1:, :-1] - K[:-1, 1:]
    sig_kers = jitted_trunc_sig_kernel(nabla, order)
    if only_last:
        return sig_kers[-1]
    else:
        return sig_kers



@njit(fastmath=True, cache=True)
def reverse_cumsum(arr:np.ndarray, axis:int): #ndim=2
    """JITed reverse cumulative sum along the specified axis.
    (np.cumsum with axis is not natively supported by Numba)"""
    A = arr.copy()
    if axis==0:
        for i in np.arange(A.shape[0]-2, -1, -1):
            A[i, :] += A[i+1, :]
    else: #axis==1
        for i in np.arange(A.shape[1]-2, -1, -1):
            A[:,i] += A[:,i+1]
    return A


@njit(fastmath=True, cache=True)
def jitted_trunc_sig_kernel(nabla:np.ndarray, # gram matrix (T_1, T_2)
                            order:int,
                            ):
    """Given difference matrix nabla_ij = K[i+1, j+1] + K[i, j] - K[i+1, j] - K[i, j+1],
    computes the truncated signature kernel of all orders up to 'order'."""
    B = np.ones((order+1, order+1, order+1, *nabla.shape))
    for d in np.arange(order):
        for n in np.arange(order-d):
            for m in np.arange(order-d):
                B[d+1,n,m] = 1 + nabla/(n+1)/(m+1)*B[d, n+1, m+1]
                r1 = reverse_cumsum(nabla * B[d, n+1, 1] / (n+1), axis=0)
                B[d+1,n,m, :-1, :] += r1[1:, :]
                r2 = reverse_cumsum(nabla * B[d, 1, m+1] / (m+1), axis=1)
                B[d+1,n,m, :, :-1] += r2[:, 1:]
                rr = reverse_cumsum(nabla * B[d, 1, 1], axis=0)
                rr = reverse_cumsum(rr, axis=1)
                B[d+1,n,m, :-1, :-1] += rr[1:, 1:]

    return B[:,0,0,0,0]


@njit((nb.float64, nb.int64), fastmath=True, cache=True)
def bessel_2sqrt(z:float,
                level:int = 10, #truncation level
           ):
    """Computes I_0( 2sqrt(z) ) where I_0 is the Bessel function of the 
    first kind, via its power series expansion truncated at 'level'."""

    out = 1
    for i in np.arange(level, 0, -1):
        out = 1 + z/i**2 * out
    return out
    

def ksig_infty(x:np.ndarray, 
            y:np.ndarray, 
            order:int, #order is truncation level of the signature
            static_kernel_gram:Callable,
            ):
    # print("x", x.shape)
    # print("y", y.shape)
    xdiff = np.diff(x, axis=0)
    ydiff = np.diff(y, axis=0)
    # print("xdiff", xdiff.shape)
    # print("ydiff", ydiff.shape)

    derivatives = static_kernel_gram(xdiff, ydiff)
    d = derivatives
    print("d", d)
    # print("derivatives", derivatives.shape)
    # return bessel_2sqrt(np.sum(derivatives), level=order)
    #TODO CHANGE HERE
    s = d[1,1] +d[0,0] + d[0,1] + d[1,0]
    return bessel_2sqrt(s, level=order)


d = 2
MAX_ORDER = 15
times_iisig = np.zeros( (MAX_ORDER) )
times_sigker  = np.zeros( (MAX_ORDER) )
times_sigpde = np.zeros( (MAX_ORDER) )
times_siginfty = np.zeros( (MAX_ORDER) )
np.random.seed(99)
T1 = 3
T2 = 3
X = np.random.randn(T1, d) / d
Y = np.random.randn(T2, d) / d
print("X", X.shape)
print("Y", Y.shape)

static_kernel = lambda X, Y : linear_kernel_gram(X, Y, custom_factor=1.0)

for order in range(1, MAX_ORDER+1):
    print("\norder", order)
    t0= time.perf_counter()
    dot1=calc_iisig_kernel(X, Y, order)
    t1 = time.perf_counter()
    dot2=trunc_sig_kernel(X, Y, order, static_kernel_gram=static_kernel, only_last=True)
    t2 = time.perf_counter()
    #dot3=calc_sigpde_kernel(X, Y)
    t3 = time.perf_counter()
    dot4=ksig_infty(X, Y, order, static_kernel_gram=static_kernel)
    t4 = time.perf_counter()
    times_iisig[order-1] = t1-t0
    times_sigker[order-1] = t2-t1
    times_sigpde[order-1] = t3-t2
    times_siginfty[order-1] = t4-t3
    print("iisig\t\t", dot1)
    print("trunc\t\t", dot2)
    print("ksig_infty\t", dot4)
    #print("dot3", dot3)




print("\ncomparison", times_iisig[1:]/times_sigker[1:])
print("\niisig", times_iisig[1:])
print("\nsigker", times_sigker[1:])
print("\npde", times_sigpde[1:])

X (3, 2)
Y (3, 2)

order 1
d [[ 0.48835662 -0.56098211]
 [ 1.21184376 -1.06183991]]
iisig		 1.0773783588346009
trunc		 1.0773783588346006
ksig_infty	 1.0773783588346006

order 2
d [[ 0.48835662 -0.56098211]
 [ 1.21184376 -1.06183991]]
iisig		 1.1595082714865306
trunc		 1.1595082714865304
ksig_infty	 1.0788752114385847

order 3
d [[ 0.48835662 -0.56098211]
 [ 1.21184376 -1.06183991]]
iisig		 1.1528540644251608
trunc		 1.1528540644251606
ksig_infty	 1.0788880807716863

order 4
d [[ 0.48835662 -0.56098211]
 [ 1.21184376 -1.06183991]]
iisig		 1.1526890212930374
trunc		 1.1526890212930372
ksig_infty	 1.0788881430096784

order 5
d [[ 0.48835662 -0.56098211]
 [ 1.21184376 -1.06183991]]
iisig		 1.1527229483873154
trunc		 1.1527229483873151
ksig_infty	 1.0788881432023134

order 6
d [[ 0.48835662 -0.56098211]
 [ 1.21184376 -1.06183991]]
iisig		 1.152722327789788
trunc		 1.1527223277897876
ksig_infty	 1.0788881432027273

order 7
d [[ 0.48835662 -0.56098211]
 [ 1.21184376 -1.06183991]]
iisig		 1.1

In [58]:
d = np.array([[ 0.48835662, -0.56098211],
              [ 1.21184376, -1.06183991]])

o1 = bessel_2sqrt(d.sum(), level=1)
print("o1", o1)
cumsum = np.cumsum(d, axis=0)
cumsum = np.cumsum(cumsum, axis=1)
cumsum = np.cumsum(cumsum, axis=0)
cumsum = np.cumsum(cumsum, axis=1)
o2 = bessel_2sqrt(cumsum.sum(), level=2) 
print("o2", o2)

o1 1.0773783600000002
o2 13.271283681174836


In [75]:
m00 = bessel_2sqrt(d[0,0], level=10)
m10 = bessel_2sqrt(d[0,0]+d[1,0], level=10)
m01 = bessel_2sqrt(d[0,0]+d[0,1], level=10)
print("m00", m00)
print("m10", m10)
print("m01", m01)

test = m10 + m01 -m00 + bessel_2sqrt(d, level=10)
print("test", test)

target = 1.1527223047952517


m00 1.5513156265384986
m10 3.574933058791967
m01 0.928682533042025
test 4.625996003011747


In [11]:
bessel_2sqrt(2, level=10)

4.2523508795013205

In [ ]:
d=9
factor = d**(1/2) * 45**(1/8)
X, Y = np.random.randn(2, 45, d) / factor
median = np.median(X, axis=0)
print("median", median)

In [ ]:
#IDEA: truncated sig --- calculate total variation and base scale on that.   TODO TODO TODO TODO

# Test variability of sig between datasets

$  \|x\|_{TV} = \sum_{i=1}^L \|x_{i}- x_{i-1}\|_{R^d} $

$ \|y\|_{R^d} = \sqrt{\sum_{k=1}^d y_k^2} $

In [ ]:
from experiments.experiment_code import normalize_streams, calc_grams



# def do_trunc_sig_gram(train, test, factor:float = 1.0):
#     ORDER = 10
#     ker = lambda X, Y: linear_kernel_gram(X, Y, param_dict["sigma"], custom_factor=factor) #TODO assumes fixed length
#     return case_truncated_sig(train, test, ORDER, 
#                                 linear_kernel_gram, sig_kernel_only_last, 
#                                 n_jobs, verbose)



def total_variation(X:np.ndarray, 
                    channelwise:bool = False,
                    mean:bool = True,
                    ):
    """Calculates the total variation of time series.
    X has shape (..., T, d)"""
    #Total Variation
    diffs = np.diff(X, axis=-2)
    if channelwise:
        TV = np.abs(diffs).sum(axis=-2) #shape (..., d)
    else:
        TV = np.linalg.norm(diffs, axis=-1).sum(axis=-1) #shape (...,)
    
    # Average
    if mean:
        ndim = TV.ndim - int(channelwise)
        if ndim > 0:
            TV = np.mean(TV, axis=tuple(range(ndim)))

    return TV


def mean_distance_between_times(X:np.ndarray, 
                                n_samples_N:int = 100,
                                n_samples_T:int = 100):
    """ X shape (N, T, d)"""

    #Sample at timesteps and instances
    N, T, d = X.shape
    n_samples_N = min(n_samples_N, N)
    n_samples_T = min(n_samples_T, T)
    choice_N = np.random.choice(N, size=n_samples_N, replace=False)
    choice_T = np.random.choice(T, size=n_samples_T, replace=False)
    X = X[choice_N][:, choice_T] #shape (n_samples_N, n_samples_T, d)

    # #out: (N, T, T))
    # new = X.transpose(1, 0, 2)
    # xx = linear_kernel_gram(new, new, diag=True, divide_by_dims=False)
    # xy = linear_kernel_gram(new, new, diag=False, divide_by_dims=False)
    # norms_squared = -2*xy + xx[:, np.newaxis] + xx[np.newaxis, :]
    # return np.mean(np.sqrt(norms_squared))

    #out: (N, T, T)) TAKE MAXIMUM ALONG T's
    new = X.transpose(1, 0, 2)
    xx = linear_kernel_gram(new, new, diag=True, divide_by_dims=False)
    xy = linear_kernel_gram(new, new, diag=False, divide_by_dims=False)
    norms_squared = -2*xy + xx[:, np.newaxis] + xx[np.newaxis, :]
    max_distances = np.max(np.sqrt(norms_squared), axis=(-1,-2))
    return np.mean(max_distances)



def test_variability(dataset_name:str):
    X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset(dataset_name)
    print(dataset_name)
    unique_labels = np.unique(y_train)
    num_classes = len(unique_labels)
    N_train, T, d = X_train.shape

    corpus, test = normalize_streams(X_train, X_test)
    print("corpus", corpus.shape)
    s = tslearn.metrics.sigma_gak(dataset=corpus,
          n_samples=100,
          random_state=0)
    
    ### TOTAL VARIATION TESTS
    # TV = total_variation(X_train, channelwise=False, mean=True)
    # print(TV)
    # ### calculate the kernel
    # choice = np.random.choice(N_train, size=20)
    # choice_test = np.random.choice(len(X_test), size=2)
    # TRAIN = np.array([corpus[i] for i in choice])
    # TEST = np.array([test[i] for i in choice_test])
    # TV_TRAIN = total_variation(TRAIN)
    # print("TV_TRAIN", TV_TRAIN)
    # TRAIN = TRAIN / TV_TRAIN * 30
    # TEST = TEST / TV_TRAIN * 30
    # print("new tv train", total_variation(TRAIN))
    # # param_dict = {"kernel_name": "gak",
    # #                 "gak_factor" : 1}
    # # param_dict = {"kernel_name": "truncated sig",
    # #                 "order" : 8}
    # param_dict = {"kernel_name": "signature pde",
    #                 "dyadic_order" : 3}
    # vv, uv = calc_grams(TRAIN, TEST, param_dict, fixed_length=True, sig_kernel_only_last=False, n_jobs=4, verbose=False)
    # print(uv.shape)
    # abs = np.mean(np.abs(uv), axis=(-1,-2))
    # print("abs", abs)
    # print("\n")
    # pass


    # ## MEAN DISTANCE TEST
    # dist = mean_distance_between_times(X_train)
    # print("dist", dist)
    # ### calculate the kernel
    # choice = np.random.choice(N_train, size=20)
    # choice_test = np.random.choice(len(X_test), size=2)
    # TRAIN = np.array([corpus[i] for i in choice])
    # TEST = np.array([test[i] for i in choice_test])
    # TRAIN = TRAIN / dist 
    # TEST = TEST / dist
    # print("new dist train", mean_distance_between_times(TRAIN))
    # # param_dict = {"kernel_name": "gak",
    # #                 "gak_factor" : 1}
    # # param_dict = {"kernel_name": "truncated sig",
    # #                 "order" : 8}
    # param_dict = {"kernel_name": "signature pde",
    #                 "dyadic_order" : 3}
    # vv, uv = calc_grams(TRAIN, TEST, param_dict, fixed_length=True, sig_kernel_only_last=False, n_jobs=4, verbose=False)
    # print(uv.shape)
    # abs = np.mean(np.abs(uv), axis=(-1,-2))
    # print("abs", abs)
    # print("\n")
    # pass


    ### calculate the kernel
    choice = np.random.choice(N_train, size=20)
    choice_test = np.random.choice(len(X_test), size=8)
    TRAIN = np.array([corpus[i] for i in choice])
    TEST = np.array([test[i] for i in choice_test])
    # param_dict = {"kernel_name": "gak",
    #                 "gak_factor" : 1}
    # param_dict = {"kernel_name": "truncated sig",
    #                 "order" : 8}
    param_dict = {"kernel_name": "signature pde",
                    "dyadic_order" : 3}
    vv, uv = calc_grams(TRAIN, TEST, param_dict, fixed_length=True, sig_kernel_only_last=False, n_jobs=4, verbose=False)
    print(uv.shape)
    abs = np.mean(np.abs(uv), axis=(-1,-2))
    print("abs", abs)
    print("\n")
    pass



for dataset_name in [
        'ArticularyWordRecognition', 
        'BasicMotions',                #skip for now, instabilities for sig linear
        'Libras',
        'NATOPS',
        'RacketSports',
        'FingerMovements',
        'Heartbeat',                   #skip for now, instabilities for sig linear
        'SelfRegulationSCP1',  
        'UWaveGestureLibrary',
        'PenDigits',
        'LSST',
        'EthanolConcentration',
        ]:
    test_variability(dataset_name)

In [ ]:
# ArticularyWordRecognition T=144, d=9
# (10, 8, 9)
# abs [1.29565776 2.27435506 2.87972308 4.07389224 5.01688708 6.38028238
#  7.11759463 7.86446547 8.08651308 8.29304982]


# BasicMotions T=100, d=6
# (10, 8, 9)
# abs [1.04759194e+00 7.15178976e+01 1.40339471e+02 4.10876657e+03
#  1.92255294e+04 2.50874495e+05 1.49520768e+06 1.27200498e+07
#  7.06710438e+07 4.39333097e+08]


# Libras T=45, d=2
# (10, 8, 9)
# abs [1.89531427 3.83885875 4.15721433 6.99827019 7.40133744 8.93425362
#  9.22978483 9.44218473 9.4780136  9.49202141]


# NATOPS T=51, d=24
# (10, 8, 9)
# abs [ 1.01604486  1.53247836  2.82986402  4.49605307  7.90352287 12.42609689
#  18.56948677 24.6321559  29.69169966 33.1874835 ]


# RacketSports T=30, d=6
# (10, 8, 9)
# abs [ 1.16522057  4.40396722  9.04935699 15.67076359 30.32986175 39.15169919
#  61.60201888 72.30843763 90.79824697 99.62780377]


# FingerMovements T=50, d=28
# (10, 8, 9)
# abs [0.97984797 1.04719032 1.04531916 1.04813437 1.04803513 1.04810413
#  1.04810097 1.048102   1.04810194 1.04810195]


# Heartbeat T=405, d=61
# (10, 8, 9)
# abs [1.08550586e+00 7.23266419e+02 9.05013282e+04 5.82884262e+06
#  8.83717077e+08 6.35578512e+10 2.92053048e+12 8.94131107e+13
#  4.99193194e+15 3.01476687e+17]


# UWaveGestureLibrary T=315, d=3
# (10, 8, 9)
# abs [  1.20473682   9.60296452  14.84277035  39.26442477  67.65798534
#  121.57540175 170.10270738 231.90516802 276.56580554 318.60532653]

$          <x, x>  + <y, y> - <x, y>  - <y, x>  =  <x-y, x> + <x-y, y> = <x-y, x-y>      $

# Plot datasets

In [ ]:
from experiments.experiment_code import normalize_streams
import plotly.express as px

def plot_dataset(dataset_name:str):
    X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset(dataset_name)
    print(dataset_name)
    unique_labels = np.unique(y_train)
    num_classes = len(unique_labels)
    N_train, T, d = X_train.shape

    corpus, test = normalize_streams(X_train, X_test)

    choice = np.random.choice(N_train, size=9)
    TRAIN = np.array([corpus[i] for i in choice])
    fig = px.line(TRAIN[0])
    fig.show()



for dataset_name in [
        'ArticularyWordRecognition', 
        'BasicMotions', 
         ###'Cricket',             # fuck cricket, too big and n_samples=10...
         ##########'ERing', #cant find dataset
        'Libras', 
        'NATOPS', 
        'RacketSports',     
        'FingerMovements',      # estimates a bit low, 10e-3
        'Heartbeat',
        'SelfRegulationSCP1',   # CAN RESAMPLE 2x or even 3x, 4x
        'UWaveGestureLibrary',
        "PenDigits",
        ""
        ]:
    plot_dataset(dataset_name)

In [ ]:
from experiments.experiment_code import normalize_streams

In [ ]:
X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset("UWaveGestureLibrary")
import plotly.express as px
print(X_train.shape)
idx = 5
fig = px.line(X_train[idx])
fig.show()
X_train, X_test = normalize_streams(X_train, X_test)
print(X_train.shape)
fig = px.line(X_train[idx])
fig.show()

In [ ]:
import plotly.express as px
X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset("SelfRegulationSCP1")
import plotly.express as px
print(X_train.shape)
idx = 2
fig = px.line(X_train[idx])
fig.show()
X_train, X_test = normalize_streams(X_train, X_test)
print(X_train.shape)
fig = px.line(X_train[idx])
fig.show()

In [ ]:
X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset("Heartbeat")
import plotly.express as px
print(X_train.shape)
idx = 2
fig = px.line(X_train[idx])
fig.show()
X_train, X_test = normalize_streams(X_train, X_test)
print(X_train.shape)
fig = px.line(X_train[idx])
fig.show()

In [ ]:
X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset("BasicMotions")
import plotly.express as px
print(X_train.shape)
idx = 8
fig = px.line(X_train[idx])
fig.show()
X_train, X_test = normalize_streams(X_train, X_test)
print(X_train.shape)
fig = px.line(X_train[idx])
fig.show()


In [ ]:
X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset("Libras")
import plotly.express as px
print(X_train.shape)
idx = 2
fig = px.line(X_train[idx])
fig.show()
X_train, X_test = normalize_streams(X_train, X_test)
print(X_train.shape)
fig = px.line(X_train[idx])
fig.show()

In [ ]:
X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset("EthanolConcentration")
import plotly.express as px
print(X_train.shape)
idx = 200
fig = px.line(X_train[idx])
fig.show()
X_train, X_test = normalize_streams(X_train, X_test)
print(X_train.shape)
fig = px.line(X_train[idx])
fig.show()

In [ ]:
X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset("HandMovementDirection")
import plotly.express as px
print(X_train.shape)
idx = 20
fig = px.line(X_train[idx])
fig.show()
X_train, X_test = normalize_streams(X_train, X_test)
print(X_train.shape)
fig = px.line(X_train[idx])
fig.show()

In [ ]:
X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset("LSST")
import plotly.express as px
print(X_train.shape)
idx = 28
fig = px.line(X_train[idx])
fig.show()
X_train, X_test = normalize_streams(X_train, X_test)
print(X_train.shape)
fig = px.line(X_train[idx])
fig.show()

In [ ]:
X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset("PenDigits")
import plotly.express as px
print(X_train.shape)
idx = 7
fig = px.line(X_train[idx])
fig.show()
X_train, X_test = normalize_streams(X_train, X_test)
print(X_train.shape)
fig = px.line(X_train[idx])
fig.show()

In [ ]:
X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset("MotorImagery")
import plotly.express as px
print(X_train.shape)
idx = 7
fig = px.line(X_train[idx])
fig.show()
X_train, X_test = normalize_streams(X_train, X_test)
print(X_train.shape)
fig = px.line(X_train[idx])
fig.show()

In [ ]:
from experiments.experiment_code import normalize_streams
import plotly.express as px
from tslearn.datasets import UCR_UEA_datasets

In [ ]:
# Epilepsy 34
# EthanolConcentration 65
# FaceDetection 2945
# FingerMovements 158
# HandMovementDirection 40
# Heartbeat 102
# LSST 176
# MotorImagery 139
# NATOPS 30
# PenDigits 749
# PEMS-SF 38
# PhonemeSpectra 85
# RacketSports 38
# SelfRegulationSCP1 134

for dataset_name in [
    "Epilepsy",
    "EthanolConcentration",
    "FaceDetection",
    "FingerMovements",
    "HandMovementDirection",
    "Heartbeat",
    "LSST",
    "MotorImagery",   #NO --- 3000 length too big, too oscillatory
    "NATOPS",
    "PenDigits",
    "PEMS-SF",
    "PhonemeSpectra",
    "RacketSports",
    "SelfRegulationSCP1",
]:
    X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset(dataset_name)
    print(dataset_name)
    print(X_train.shape)
    idx=0
    px.line(X_train[idx]).show()
    X_train, X_test = normalize_streams(X_train, X_test)
    print(X_train.shape)
    px.line(X_train[idx]).show()
